In [ ]:
import tensorflow as tf
import pandas as pd
from collections import Counter
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import normalize
import os

In [20]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
config = tf.ConfigProto()
config.gpu_options.allow_growth=True

## Loading Data

In [66]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer

# laod data with pandas
cols = ['user', 'item', 'rating', 'timestamp']
train = pd.read_csv('ml-100k/ua.base', delimiter='\t', names=cols)
test = pd.read_csv('ml-100k/ua.test', delimiter='\t', names=cols)

In [67]:
train.head()

,user,item,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [248]:
n_users = max(train['user']); n_items = max(train['item']); n_ratings_train = len(train); n_ratings_test = len(test)

In [296]:
whole = pd.concat([train, test])
whole_sorted_by_user_time = whole.sort_values(['user', 'timestamp'],ascending=True)

In [297]:
Last_movie = list()
for i in range(n_users):
    Last_movie.append(whole_sorted_by_user_time[whole_sorted_by_user_time['user'] == i+1].tail(1).iloc[0,1])

In [298]:
n_columns = n_users+2*n_items+1
n_rows_train = n_ratings_train; n_rows_test = n_ratings_test
X_train = np.zeros((n_rows_train, n_columns)); X_test = np.zeros((n_rows_test, n_columns))
Y_train = np.zeros((n_rows_train, 1)); Y_test = np.zeros((n_rows_test, 1))

In [299]:
for i in range(n_ratings_train):
        index_user = train.iloc[i,0] - 1
        index_item = train.iloc[i,1] - 1
        index_last_item = Last_movie[index_user]
        score = train.iloc[i,2]
        time_stamp = train.iloc[i,3]
        readable = datetime.datetime.fromtimestamp(time_stamp).isoformat()
        month = readable[5:7]
        X_train[i,index_user] = 1; X_train[i,n_users+index_item] = 1; X_train[i,n_users+n_items+index_last_item] = 1 
        X_train[i,n_columns-1] = month; Y_train[i] = score
for i in range(n_ratings_test):
        index_user = test.iloc[i,0] - 1
        index_item = test.iloc[i,1] - 1
        index_last_item = Last_movie[index_user]
        score = test.iloc[i,2]
        time_stamp = test.iloc[i,3]
        readable = datetime.datetime.fromtimestamp(time_stamp).isoformat()
        month = readable[5:7]
        X_test[i,index_user] = 1; X_test[i,n_users+index_item] = 1; X_test[i,n_users+n_items+index_last_item] = 1 
        X_test[i,n_columns-1] = month; Y_test[i] = score

In [300]:
X_train = normalize(X_train)
X_test = normalize(X_test)

In [301]:
_, n = X_train.shape

# number of latent factors
k = 40

# design matrix
X = tf.placeholder('float', shape=[None, n])
# target vector
y = tf.placeholder('float', shape=[None, 1])

# bias and weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([n]))

# interaction factors, randomly initialized 
V = tf.Variable(tf.random_normal([k, n], stddev=0.01))


In [302]:
# Calculate output with FM equation
linear_terms = tf.add(w0, tf.reduce_sum(tf.multiply(W, X), 1, keep_dims=True))
pair_interactions = (tf.multiply(0.5,
                    tf.reduce_sum(
                        tf.subtract(
                            tf.pow( tf.matmul(X, tf.transpose(V)), 2),
                            tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2)))),
                        1, keep_dims=True)))
y_hat = tf.add(linear_terms, pair_interactions)

In [303]:
# L2 regularized sum of squares loss function over W and V
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_w, tf.pow(W, 2)),
                tf.multiply(lambda_v, tf.pow(V, 2)))))

error = tf.square(tf.subtract(y, y_hat))
cost = tf.reduce_mean(error)
loss = tf.add(cost, l2_norm)

In [304]:
training_truth = tf.less(tf.abs(error), 0.5)
acc = tf.reduce_mean(tf.cast(training_truth, tf.float32))

In [305]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

In [306]:
epochs = 30

# Launch the graph
init = tf.global_variables_initializer()
sess = tf.Session(config=config)

sess.run(init)

for epoch in range(epochs):
    sess.run(optimizer, feed_dict={X: X_train, y: Y_train})
    cost_train = sess.run(cost, feed_dict={X: X_train, y: Y_train})
    cost_val = sess.run(cost, feed_dict={X: X_test, y: Y_test})
    acc_val = sess.run(acc, feed_dict={X: X_test, y: Y_test})
    print("epoch ",epoch,":\n")
    print("train Cost : ", cost_train)
    print("Val Cost : ", cost_val)
    print("Validation Accuracy : ", acc_val)
    print('****************************************** \n')

epoch  0 :

train Cost :  11.574
Val Cost :  12.064
Validation Accuracy :  0.0268293
****************************************** 

epoch  1 :

train Cost :  9.03173
Val Cost :  9.85958
Validation Accuracy :  0.0590668
****************************************** 

epoch  2 :

train Cost :  6.35791
Val Cost :  7.4142
Validation Accuracy :  0.0910923
****************************************** 

epoch  3 :

train Cost :  4.29541
Val Cost :  5.1397
Validation Accuracy :  0.167232
****************************************** 

epoch  4 :

train Cost :  3.40533
Val Cost :  3.82698
Validation Accuracy :  0.229374
****************************************** 

epoch  5 :

train Cost :  2.88252
Val Cost :  3.15791
Validation Accuracy :  0.255885
****************************************** 

epoch  6 :

train Cost :  2.42663
Val Cost :  2.79208
Validation Accuracy :  0.267444
****************************************** 

epoch  7 :

train Cost :  2.18451
Val Cost :  2.66909
Validation Accuracy :  0.26214

In [309]:
RMSE = np.sqrt(np.array(cost_train).mean())

In [310]:
RMSE

0.98294288

In [311]:
y_pred = sess.run(y_hat, feed_dict={X: X_train, y: Y_train})

In [312]:
y_pred[0:10]

array([[ 3.69119453],
       [ 3.42851424],
       [ 3.39970422],
       [ 3.54519677],
       [ 3.2105372 ],
       [ 3.43632531],
       [ 3.65410852],
       [ 3.72488856],
       [ 3.63556552],
       [ 3.61127949]], dtype=float32)

In [313]:
Y_train[0:10]

array([[ 5.],
       [ 3.],
       [ 4.],
       [ 3.],
       [ 3.],
       [ 5.],
       [ 4.],
       [ 1.],
       [ 5.],
       [ 3.]])